In [56]:
import os
import shutil
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import os
import cv2
import shutil
from scipy.stats import mannwhitneyu


In [57]:
root = 'results/params/'
for run in os.listdir(root):
    if run == '0images':
        continue
    for folder in tqdm(os.listdir(root + run)):
        for file in os.listdir(root + run + '/' + folder):
            if file.endswith('.json'):
                number = file.replace('.json', '').replace('config-', '')
                try:
                    shutil.move(root + 'fed-job-{}.out'.format(number), root + folder)
                except:
                    pass
            if file.endswith('npy'):
                metric = file.replace('.npy', '')
                if os.path.exists(root + run + '/' + folder + '/' + metric + '.png'):
                    continue
                y = np.load(root + run + '/' + folder + '/' + file)
                x = np.arange(1, len(y) + 1)
                mean = y.cumsum() / x
                fig = plt.figure()
                plt.plot(x, y, label=metric.replace('_', ' '))
                plt.plot(x, mean, label=metric.replace('_', ' ') + ' mean')
                plt.xlabel('num of round')
                plt.grid(True)
                plt.yticks(np.arange(0, 1, 0.1))
                plt.legend()
                plt.savefig(root + run + '/' + folder + '/' + metric + '.png', bbox_inches='tight')
                plt.close(fig)

100%|██████████| 7/7 [00:00<00:00, 10065.18it/s]


In [58]:
krum_images = []


def load(root):
    result_map = defaultdict(list)
    for run in os.listdir(root):
        if run == '0images':
            continue
        for folder in os.listdir(root + run):
            if folder == '0images':
                continue
            accuracy_m = 0
            accuracy_s = 0
            for file in os.listdir(root + run + '/' + folder):
                if file.endswith('.npy'):
                    results = np.load(root + run + '/' + folder + '/' + file)
                    num = len(results) // 10
                    accuracy_m = np.median(results[-num:])
                    deg = '0.4'
                    if 'adni' in folder:
                        deg = '0'
                    if deg + '--krum--clean' in folder:
                        krum_images.append(root + run + '/' + folder + '/' + file.replace('.npy', '.png'))
                    accuracy_s = results[-num:].std()
                    if file.startswith('backdoor'):
                        break
            result_map[folder].append((accuracy_m, accuracy_s))
    return result_map


result_map = load(root)

path = root + '0images/krum/'
os.makedirs(path, exist_ok=True)
for i, image in enumerate(krum_images):
    shutil.copy(image, path + '{}.png'.format(i))



In [59]:
data = []
for config in result_map:
    results = result_map[config]
    results = np.median(results, axis=0)
    a = list(filter(lambda x: x, config.split('-')))
    dataset = a[0]
    non_iid = a[1]
    aggregator = a[2]
    attack = a[3]
    fraction = a[4] if len(a) > 4 else 0
    parameter_name = a[5] if len(a) > 5 else ''
    parameter_val = a[6] if len(a) > 6 else 0
    data.append((dataset, non_iid, aggregator, attack, fraction, parameter_name, parameter_val, *results))

df = pd.DataFrame(data,
                  columns=['dataset', 'non_iid', 'aggregator', 'attack', 'fraction', 'parameter_name', 'parameter_val',
                           'accuracy_m', 'accuracy_s'])

df.to_csv(root + '0images/raw.csv')

In [60]:

def autolabel(rects):
    """
    Attach a text label above each bar displaying its height
    """
    for rect in rects:
        height = rect.get_height()
        plt.text(rect.get_x() + rect.get_width() / 2., 1.03 * height,
                 '%d' % int(np.round(height)),
                 ha='center', va='bottom')



In [86]:
# clean = df[df['attack'] == 'clean']
attacks = df[df['attack'] != 'clean']

for mutator, group in attacks.groupby('attack'):
    print(mutator)
    group = group.sort_values('parameter_val')
    values = group['accuracy_m'].to_numpy() * 100
    # values = np.concatenate((clean['accuracy_m'].to_numpy(), values)) * 100
    x = group['parameter_val'].tolist()
    y_label = 'Accuracy'

    plt.rcParams.update({'font.size': 22})
    fig = plt.figure(figsize=(10, 6))
    autolabel(plt.bar(x, values, .35, color='darkcyan'))
    x1, x2, y1, y2 = plt.axis()
    plt.axis((x1 - .5, x2 + .5, y1, y2 + 8))
    plt.ylabel(y_label)
    plt.xlabel(group['parameter_name'].tolist()[0].replace('_', ' '))
    plt.savefig(root + '0images/' + mutator + '.png', bbox_inches='tight')
    plt.close(fig)
    # plt.show()

deletedata
noisedata
overlapdata
unbalancedata
